In [1]:
# Imports 

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mticker 
from matplotlib import cm
import os 
import itertools 
import mesa_reader as mr 
import time 

import src 

# Automatically reload modules 
%load_ext autoreload 
%autoreload 2 

# Make matplotlib plots open in a separate interactive window 
%matplotlib qt 



In [ ]:
# To do: 






# Research questions 

# Look in a 6 and 10 solar mass star, the carbon and nitrogen abundances after helium fusion has ignited. 
# At some point, nitrogen should plummet while carbon stays at the same level. 
# C+N should go down. Normally is constant because you're just turning C into N, but now we're depleting the N as well. 
# High temp: C+N rises because you're creating C and there is no N 
# There should be a thin layer that used to be in the core but isn't anymore. C+N will drop. 











In [ ]:
# Testing profile or history plots  

history = load_data.load_history(stellar_evolution_data.data_folder/"M=1.0")
# profile = load_data.load_profile(stellar_evolution_data.data_folder/"M=1.0", modelnum=300, history=history) 

# _ = profile_plotting.ProfilePlot.temp(profile, history=history) 
# _ = profile_plotting.ProfilePlot.degeneracy(profile, history=history) 
# _ = profile_plotting.ProfilePlot.fusion(profile, history=history) 
# _ = profile_plotting.ProfilePlot.mu(profile, history=history) 
# _ = profile_plotting.ProfilePlot.composition(profile, history=history) 
# _ = history_plotting.HistoryPlot.composition(history) 



In [7]:



# Initialize plot 
hr = src.plot.hr.hr.HRDiagram()  

for mass in [0.2, 0.4, 0.5, 0.65, 1.0, 3.0]: 
    history = src.load_data.load_history(src.data.file_paths.MESA_data_folder/f"M={mass}")
    hr.add_path(history, label=mass) 

hr.legend(fontsize=12, loc="center left", bbox_to_anchor=(1, 0.5)) 
hr.add_spectral_type_labels()  




In [ ]:
# Test lag 




# results = []
# for _ in range(10):







#     # Initialize plot 
#     hr = HRDiagram() 

#     for mass in [0.2, 0.4, 0.5, 0.65, 1.0, 3.0]: 
#         history = load_data.load_history(stellar_evolution_data.data_folder/f"M={mass}")
#         hr.add_path(history, label=mass) 

#     hr.ax.legend() 
#     # hr.toggle_grid_during_pan()

#     ax = hr.ax 
#     fig = hr.fig 




#     times = []

#     def on_draw(event):
#         times.append(time.perf_counter())

#     cid = fig.canvas.mpl_connect("draw_event", on_draw)

#     sleeptime_ms = 50

#     # Run automated pan-like updates
#     for i in range(50): 
#         current_xlim = ax.get_xlim() 
#         ax.set_xlim(current_xlim[0]*(1.01), current_xlim[1]*(1.01))
#         fig.canvas.draw_idle()
#         fig.canvas.flush_events()
#         time.sleep(sleeptime_ms/1000)

#     plt.close(fig)

#     # Compute time differences
#     times = np.array(times)*1000 # millisconds 
#     dt = np.diff(times)-sleeptime_ms
#     print(_)
#     print(f"Mean: {np.mean(dt):.4f}")
#     print(f"Median: {np.median(dt):.4f}")
#     print(f"STD: {np.std(dt):.4f}")
#     print(f"Max: {np.max(dt):.4f}")
#     print(f"50th percentile (median): {np.percentile(dt, 50):.4f}")
#     print(f"90th percentile: {np.percentile(dt, 90):.4f}")
#     print(f"99th percentile: {np.percentile(dt, 99):.4f}") 
#     print("\n")

#     results.append(np.median(dt)) 

# print(f"Median of medians: {np.median(results):.1f}")
# print(f"STD of medians:     {np.std(results):.1f}")


In [ ]:
# Nothing plotted 
# Median of medians: 64.2
# STD of medians:     2.8

# Full path 
# Median of medians: 65.7
# STD of medians:     4.5

# Simplified path 
# Median of medians: 64.6
# STD of medians:     2.4

# 10 full plots 
# Median of medians: 126.8
# STD of medians:     8.1 

# 10 simple plots 
# Median of medians: 94.4
# STD of medians:     3.6

# for mass in [0.2, 0.4, 0.5, 0.65, 1.0, 3.0]: (Simple)
# Median of medians: 87.6
# STD of medians:     6.0 

# for mass in [0.2, 0.4, 0.5, 0.65, 1.0, 3.0]: (Full)
# Median of medians: 96.6
# STD of medians:     8.3

In [26]:
folder = stellar_evolution_data.data_folder/"Mass=1.75_models=every5" 
history = load_data.load_history(folder) 



plt.figure(figsize=(10,8))
plt.plot(history.star_age, history.he_core_mass, label="Helium", color="tab:green")
plt.plot(history.star_age, history.c_core_mass, label="Carbon", color="tab:red")
plt.plot(history.star_age, history.o_core_mass, label="Oxygen", color="tab:brown")
plt.plot(history.star_age, history.fe_core_mass, label="Iron", color="black")
plt.ylabel("Mass")
plt.legend() 

plt.figure(figsize=(10,8))
plt.plot(history.star_age, history.he_core_radius, label="Helium", color="tab:green")
plt.plot(history.star_age, history.c_core_radius, label="Carbon", color="tab:red")
plt.plot(history.star_age, history.o_core_radius, label="Oxygen", color="tab:brown")
plt.plot(history.star_age, history.fe_core_radius, label="Iron", color="black")
plt.ylabel("Radius")
plt.legend() 




In [ ]:
# Test gradient rectangle 


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import blended_transform_factory

fig, ax = plt.subplots(figsize=(8, 4))

# Example background plot
ax.plot([0, 1, 2, 3], [0, 1, 0, 1], color='gray')

# Define where your gradient bar goes
xmin, xmax = 1, 2
height = 0.1
y_bottom = 1.0  # top of the plot
trans = blended_transform_factory(ax.transData, ax.transAxes)

# Create a gradient array from white → color → white
n = 256
color = np.array([1.0, 0.3, 0.3])  # your base color (e.g. red)
white = np.ones(3)
gradient = np.linspace(0, 1, n)
rgb = np.outer(1 - np.abs(gradient * 2 - 1), color) + np.outer(np.abs(gradient * 2 - 1), white)

# rgb shape (n,3) → (1,n,3) for imshow
gradient_img = rgb[np.newaxis, :, :]

# Display gradient as image in data coordinates
ax.imshow(
    gradient_img,
    extent=(xmin, xmax, y_bottom, y_bottom + height),
    transform=trans,
    aspect='auto',
    clip_on=False,
    zorder=3,
)

ax.set_xlim(0, 3)
ax.set_ylim(0, 1.5)
plt.show()


In [34]:
# Look for boundaries between substages 

# To do: 0.2, 0.4, 3.0 
# 0.2 could have a mini-red giant phase before the WD phase 





folder = src.data.file_paths.MESA_data_folder/"M=3.0" 
history = src.load_data.load_history(folder)
print(history.model_numbers_available)
modelnum = 11000  


# HR diagram 
hr = src.plot.hr.hr.HRDiagram()
hr.add_path(history, label=history.initial_mass_string)
hr.ax.scatter(10**history.log_Teff[modelnum-1], 10**history.log_L[modelnum-1], color="red", ec="black", zorder=100)  
plt.legend() 


# History plots 
# _ = src.plot.history.HistoryPlot.composition(history, modelnum_now=modelnum)  
# _ = src.plot.history.HistoryPlot.fusion(history, modelnum_now=modelnum)  
_ = src.plot.history.HistoryPlot.radius(history, modelnum_now=modelnum)  
# _ = src.plot.history.HistoryPlot.mass(history, modelnum_now=modelnum)  


# Interior composition 
if modelnum in history.model_numbers_available: 
    profile = src.load_data.load_profile(folder, modelnum=modelnum, history=history) 


# Profile plots 
src.plot.profile.profile.ProfilePlot.composition(profile, history=history) 
# src.plot.profile.profile.ProfilePlot.convection(profile, history=history) 
# src.plot.profile.profile.ProfilePlot.fusion(profile, history=history) 


# Print info 
print(f"model = {modelnum}") 
print(f"age = {history.star_age[modelnum-1]}") 







[    1    50   100   150   200   225   250   268   300   313   348   350
   363   400   433   450   500   548   550   600   613   650   700   750
   800   850   900   950  1000  1050  1100  1150  1200  1250  1300  1350
  1400  1450  1500  1550  1600  1650  1700  1750  1800  1850  1900  1950
  2000  2050  2100  2150  2200  2250  2300  2350  2400  2450  2500  2550
  2600  2650  2700  2750  2800  2850  2900  2950  3000  3050  3100  3150
  3200  3250  3300  3350  3400  3450  3500  3550  3600  3650  3700  3750
  3800  3850  3900  3950  4000  4050  4100  4150  4200  4250  4300  4350
  4400  4450  4500  4550  4600  4650  4700  4750  4800  4850  4900  4950
  5000  5050  5100  5150  5200  5250  5300  5350  5400  5450  5500  5550
  5600  5650  5700  5750  5800  5850  5900  5950  6000  6050  6100  6150
  6200  6250  6300  6350  6400  6450  6500  6550  6600  6650  6700  6750
  6800  6850  6900  6950  7000  7050  7100  7150  7200  7250  7300  7350
  7400  7450  7500  7550  7600  7650  7700  7750  7

In [ ]:
# 0.2 M_sun (represents 0.1 - 0.3) 
# Fully convective 

# Middle of hayashi track (already available in stellar_evolution_data.data_folder/"M=0.2") 
# model = 150
# age = 733779.8346512254 

# Middle of MS (already available in stellar_evolution_data.data_folder/"M=0.2")** 
# model = 273 
# age = 462063191253 

# Middle of He WD (already available in stellar_evolution_data.data_folder/"M=0.2")
# model = 1200
# age = 1026553681667.4374

In [ ]:
# 0.4 M_sun (represents 0.3 - 0.5) 
# No henyey track, not fully convective, never fuses helium  

# Middle of Hayashi track (already available in stellar_evolution_data.data_folder/"M=0.4") 
# model = 200
# age = 1657716.2443819284  

# Middle of MS (already available in stellar_evolution_data.data_folder/"M=0.4") 
# model = 309
# age = 72844865022.73999 

# Subgiant (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 450? 

# Red giant (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 3000? 

# Helium white dwarf (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 5159 




In [ ]:
# 1.0 M_sun (represents 0.5 - 1.5) 

# Hayashi 
# start = 1 
# model = 150
# end = 202 

# Henyey 
# start = 202
# model = 220
# end = 240  

# MS 
# start = 240
# model = 296
# end = 330

# Subgiant 
# start = 330 
# model = 389 
# end = 415 

# Red giant 
# start = 415 
# model = 5000
# end = 9500 

# Helium flash 
# start = 9500 
# model = 9700 
# end = 10500 

# He MS 
# start = 10500 
# model = 10650
# end = 10950 

# AGB 
# start = 10950 
# model = 12300 
# end = 13600 

# On the way to Helium white dwarf 
# start = 13600 
# model = 14300
# end = 14300 

In [ ]:
# 1.75 M_sun (represents 1.5 - 2) 
# Hertzsprung gap, but helium flash? 

# Hayashi 
# start = 1 
# model = 140 
# end = 200 

# Henyey 
# start = 200 
# model = 235 
# end =  250 

# MS 
# start = 250 
# model = 285 
# end = 340 

# Hertzsprung gap  
# start = 340 
# model = 389 
# end = 420 

# Red giant 
# start = 420 
# model = 1000 
# end = 8763

# Helium flash 
# start =  8763
# model = 8802
# end = 9000 

# He MS 
# start = 9000 
# model = 9800 
# end =  10000

# AGB 
# start =  10000
# model =  11425
# end =  13635

# On the way to Helium white dwarf 
# start =  
# model = 
# end = 

In [ ]:
# 3 M_sun (represents 2 - 6) 

# Hayashi (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 150 

# Henyey (already available in stellar_evolution_data.data_folder/"M=3.0")** 
# model = 225
# age = 2063039.3885299314 

# MS (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 300 

# Hertzsprung Gap (already available in stellar_evolution_data.data_folder/"M=3.0")** 
# model = 363 
# age = 322089285.1496673

# Red giant (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 400 

# Helium stable ignition 

# Helium main sequence (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 650?  

# AGB (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 1700

# C+O white dwarf 
# Need to extend age!! 



In [196]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate moment of inertia  
def calc_inertia(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass)
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    radius_coord_sorted_cm = profile.radius[ind_sort] * physical_constants.R_sun
    
    # I = integral of 2/3 * r**2 over the mass coordinate (2/3 factor because its the I of a spherical shell)
    inertia = np.trapz(2/3* radius_coord_sorted_cm**2, x=mass_coord_sorted_g)
    
    return inertia 


ages = [] 
inertias = [] 
m_rsquareds = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    inertias.append(calc_inertia(profile))
    m_rsquareds.append(history.star_mass[modelnum-1]*physical_constants.M_sun * ((10**history.log_R[modelnum-1])*physical_constants.R_sun)**2)




# Plot moment of inertia and MR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, m_rsquareds, label="MR^2", lw=3) 
plt.plot(ages, inertias, label="Moment of inertia", lw=3) 
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("grams * cm^2") 
title = f"Moment of Inertia vs time of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")


# Plot ratio of I to MR^2 to get the f-constant in I=fMR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(inertias)/np.array(m_rsquareds), lw=3) 
plt.xlabel("Age (years)")  
title = f"f vs time for a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")








1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [ ]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate internal energy 
def calc_internal_energy(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass) 
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    temp_sorted = 10**profile.logT[ind_sort] 
    mu_sorted = profile.mu[ind_sort]

    # E = 3/2 * kT / (mu*m_p) 
    energy = np.trapz(3/2 * physical_constants.k * temp_sorted / (mu_sorted*physical_constants.m_p), x=mass_coord_sorted_g)
    return energy 



ages = [] 
energies = [] 
g_msquared_over_r = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    energies.append(calc_internal_energy(profile))
    g_msquared_over_r.append(physical_constants.G * (history.star_mass[modelnum-1]*physical_constants.M_sun)**2 / ((10**history.log_R[modelnum-1])*physical_constants.R_sun))




# Plot GM^2 / R 
plt.figure(figsize=(15, 8))
plt.plot(ages, g_msquared_over_r, label="GM^2 / R", lw=3) 
plt.plot(ages, energies, label="3/2*kT/(mu*m_p)", lw=3)
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("ergs") 
title = f"Gravitational and Thermal Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")



# Plot ratio 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(energies)/np.array(g_msquared_over_r), lw=3) 
plt.ylim((0.5, 1.4))
plt.xlabel("Age (years)") 
title = f"Ratio of Thermal Energy to Gravitational Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")







1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [312]:
# Test colormaps 


# potential_colormaps = [32, 42, 47, 49, 52, 53, 54, 57, 60, 61, 62, 65, 66, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83] 

for cmap_name in ["cubehelix", "gist_earth", "gist_ncar", "gist_rainbow", "jet", "nipy_spectral", "rainbow", "Spectral"]:
    subdivisions = [8, 10, 14, 20, 500] 

    fig, axes = plt.subplots(nrows=len(subdivisions), figsize=(12, 9))
    for ax, n in zip(axes, subdivisions):

        cmap = cm.get_cmap(cmap_name, n)

        # Plot color using axvspan 
        for i in range(n): 
            ax.axvspan(i-1/2, i+1-1/2, color=cmap(i))

        # Add text to xtick labels to show what command to use to access that color 
        ax.xaxis.set_major_locator(mticker.MaxNLocator(integer=True))
        def label_formatter(x, pos):
            return f"cmap({int(x)})" 
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(label_formatter))

        # Set title to show what command to use to get this colormap, set x limit, and disable y axis ticks 
        ax.set_title(f"cmap = cm.get_cmap(\"{cmap_name}\", {n})") 
        ax.set_xlim(-1/2, n-1/2) 
        ax.yaxis.set_visible(False)

    plt.tight_layout() 
    break 
    # plt.savefig(f"{cmap_name}.jpg")




C:\Users\johnm\AppData\Local\Temp\ipykernel_5088\2247804312.py:11: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name, n)
